In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

from PIL import Image
import keras

In [ ]:
HEIGHT = 224
WIDTH = 224

base_model = MobileNetV2(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

In [ ]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers[:-4]:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import preprocess_input


TRAIN_DIR = "../input/currency-train-test/train_test/train_data"
HEIGHT = 224
WIDTH = 224
BATCH_SIZE = 8

train_datagen =  ImageDataGenerator(
       preprocessing_function=preprocess_input,
       zoom_range = 0.3,
       rotation_range=90,
       horizontal_flip=True,
       vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)
    
    

In [ ]:
TEST_DIR = "../input/currency-train-test/train_test/test_data"
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)


In [ ]:
FC_LAYERS = [128]
dropout = 0.4

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=10)

In [ ]:
from keras.optimizers import SGD, Adam

NUM_EPOCHS = 10
BATCH_SIZE = 8
num_train_images = 4206
num_test_images = 1232


adam = Adam(lr=0.0001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                             steps_per_epoch=num_train_images // BATCH_SIZE,
                             validation_steps=num_test_images // BATCH_SIZE,
                             validation_data = test_generator)


In [ ]:
finetune_model.fit_generator(train_generator, epochs=5, workers=8, 
                             steps_per_epoch=num_train_images // BATCH_SIZE,
                             validation_steps=num_test_images // BATCH_SIZE,
                             validation_data = test_generator)

In [ ]:
finetune_model.fit_generator(train_generator, epochs=7, workers=8, 
                             steps_per_epoch=num_train_images // BATCH_SIZE,
                             validation_steps=num_test_images // BATCH_SIZE,
                             validation_data = test_generator)

In [ ]:
adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

finetune_model.fit_generator(train_generator, epochs=10, workers=8, 
                             steps_per_epoch=num_train_images // BATCH_SIZE,
                             validation_steps=num_test_images // BATCH_SIZE,
                             validation_data = test_generator)

In [ ]:
adam = Adam(lr=0.000001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
finetune_model.fit_generator(train_generator, epochs=5, workers=8, 
                             steps_per_epoch=num_train_images // BATCH_SIZE,
                             validation_steps=num_test_images // BATCH_SIZE,
                             validation_data = test_generator)

In [ ]:
finetune_model.fit_generator(train_generator, epochs=5, workers=8, 
                             steps_per_epoch=200,
                             validation_steps=num_test_images // BATCH_SIZE,
                             validation_data = test_generator)

In [ ]:
from keras.models import load_model

finetune_model.save('/kaggle/working/my_model1.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
from keras.models import load_model
model = load_model('my_model1.hdf')

no_to_note={0:"100",1:"50",2:"10",3:"2000",4:"200",5:"20",6:"500",7:"50",8:"100",9:"No note"}
img="F:/Capstone Project/Currency recognition/currency_dataset/20_old/319.jpg"

import cv2,numpy as np

img = Image.open(img)
img = np. array(img)
img=cv2.resize(img,(224,224))
img=preprocess_input(img)
#from matplotlib import pyplot as plt
#plt.imshow(img, interpolation='nearest')
#plt.show()

img = np.expand_dims(img, axis=0)

arr=model.predict(img)
print(arr[0][0])

mx,ind=arr[0][0],0
for i in range(10):
    if(arr[0][i]>mx):
        mx,ind=arr[0][i],i
        
print(no_to_note[ind])    

In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks('.') #lists all downloadable files on server